<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 3: Feature Engineering

## Using Explode, User Defined Functions, and Pivot


### Lesson Objectives: 

- After completing this lesson, you should be able to use these methods on DataFrames:
-	`explode()`
-	User Defined Functions 
-	`pivot()`



### How Can We Turn Sales Data Into A Report

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

spark = org.apache.spark.sql.SparkSession@15c181c0


org.apache.spark.sql.SparkSession@15c181c0

In [2]:
// Setting Up the Example

case class Sales( id: Int, account: String, year: String, commission: Int, sales_reps: Seq[String])


defined class Sales


In [3]:
val sales = spark.createDataFrame(Seq(
  Sales(1, "Acme", "2013", 1000, Seq("Jim", "Tom")),
  Sales(2, "Lumos", "2013", 1100, Seq("Fred", "Ann")),
  Sales(3, "Acme", "2014", 2800, Seq("Jim")),
  Sales(4, "Lumos", "2014", 4200, Seq("Fred", "Sally"))
))
sales.show()

+---+-------+----+----------+-------------+
| id|account|year|commission|   sales_reps|
+---+-------+----+----------+-------------+
|  1|   Acme|2013|      1000|   [Jim, Tom]|
|  2|  Lumos|2013|      1100|  [Fred, Ann]|
|  3|   Acme|2014|      2800|        [Jim]|
|  4|  Lumos|2014|      4200|[Fred, Sally]|
+---+-------+----+----------+-------------+



sales = [id: int, account: string ... 3 more fields]


[id: int, account: string ... 3 more fields]

## explode()

In [4]:
sales.select( $"id", $"account", $"year", $"commission", explode($"sales_reps") as("sales_rep")).show()

+---+-------+----+----------+---------+
| id|account|year|commission|sales_rep|
+---+-------+----+----------+---------+
|  1|   Acme|2013|      1000|      Jim|
|  1|   Acme|2013|      1000|      Tom|
|  2|  Lumos|2013|      1100|     Fred|
|  2|  Lumos|2013|      1100|      Ann|
|  3|   Acme|2014|      2800|      Jim|
|  4|  Lumos|2014|      4200|     Fred|
|  4|  Lumos|2014|      4200|    Sally|
+---+-------+----+----------+---------+



## User Defined Functions

In [5]:
val len: (Seq[String] => Int) = (arg: Seq[String]) => {arg.length}
val column_len = udf(len)


len = > Int = <function1>
column_len = UserDefinedFunction(<function1>,IntegerType,Some(List(ArrayType(StringType,true))))


UserDefinedFunction(<function1>,IntegerType,Some(List(ArrayType(StringType,true))))

In [6]:
val exploded = sales.select( $"id", $"account", $"year", $"commission", ($"commission" / column_len($"sales_reps")).as("share"), explode($"sales_reps").as("sales_rep"))
exploded.show()

+---+-------+----+----------+------+---------+
| id|account|year|commission| share|sales_rep|
+---+-------+----+----------+------+---------+
|  1|   Acme|2013|      1000| 500.0|      Jim|
|  1|   Acme|2013|      1000| 500.0|      Tom|
|  2|  Lumos|2013|      1100| 550.0|     Fred|
|  2|  Lumos|2013|      1100| 550.0|      Ann|
|  3|   Acme|2014|      2800|2800.0|      Jim|
|  4|  Lumos|2014|      4200|2100.0|     Fred|
|  4|  Lumos|2014|      4200|2100.0|    Sally|
+---+-------+----+----------+------+---------+



exploded = [id: int, account: string ... 4 more fields]


[id: int, account: string ... 4 more fields]

## pivot()

In [7]:
exploded.groupBy($"sales_rep").pivot("year").agg(sum("share")).orderBy("sales_rep").collect

Array([Ann,550.0,null], [Fred,550.0,2100.0], [Jim,500.0,2800.0], [Sally,null,2100.0], [Tom,500.0,null])

In [8]:
exploded.groupBy($"account", $"sales_rep").pivot("year").agg(sum("share")).orderBy("account", "sales_rep").collect

Array([Acme,Jim,500.0,2800.0], [Acme,Tom,500.0,null], [Lumos,Ann,550.0,null], [Lumos,Fred,550.0,2100.0], [Lumos,Sally,null,2100.0])

### Lesson Summary

Having completed this lesson, you should understand the role of

-	`explode()`
-	User Defined Functions 
-	`pivot()`


### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.